## Task REDACTED 

Analysis plan
-	Data cleaning 

o	Screen for outliers

o	Inconsistencies in coding e.g. UK or United Kingdom

o	Outliers – Extreme cases 

o	Multivariate outliers – who they are and why they’re problematic

o	Normality or non-normality

IRs 
-	Is there a correlation between score & sentiment_score if so can we predict? (IR COMPLETE) r(7326)0.545 p<.001.
-   Can we predict score & sentiment_score?                                                     (IR COMPLETE)
-	Where are most of these coming from? Which country? – Pie Chart                             (IR COMPLETE)
-	When were most of these interactions? - Histogram                                           (IR COMPLETE) 
-	What modality are the interactions?  - Pie Chart                                            (IR COMPLETE)
-	How long are these interactions open for? – open date v close date
-	Are there repeat interactions? – histogram for interaction ID I guess - Not interested anymore
-	Do a histogram of the client priority – not sure what it is but it might be which clients   (IR COMPLETE)
-	Is satisfaction different if they’re at different priorities? - Group by priority, DV = score
-	Do different regions score us differently? - Not enough time - just go globally
-	Break data down by client – we can also look at their common modalities - Not enough time
-	Does priority affect score and sentiment_score? - Breakdown by priority, DV == Sentiment score

Analysis of comments – I am not going through all of these. So we’ll select from the following sub-samples
-	Those who score in the lower quartile for score – what are they saying?
-	Those who score in the upper quartile for score – what are they saying? 
-	Breakdown by touchpoint – If people are going to a particular touch point – what are they saying? 
-	Breakdown by client – what are different clients saying?
-	Breakdown by length – upper and lower quartiles – are there differences in what they’re saying? 

Points to bare in mind or decisions to make
- Do we focus on one region?
- Do we focuc on specific clients?

Extra IR
- Were the scores and sentient_scores worse in March when the most incidents were occurring? Is it possible that we were overwhelmed? 


In [ ]:
#Import dataset
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
import seaborn as sns
from scipy.stats import shapiro
from scipy.stats import spearmanr
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy.stats.mstats import winsorize
from scipy.stats import chi2
import statsmodels.api as sm
from scipy.stats import kstest
from scipy.stats import wilcoxon
import statsmodels.formula.api as smf
from scipy.stats import levene
import statsmodels.api as sm

In [ ]:
#Import data
os.chdir('REDACTED')
df = pd.read_excel('REDACTED')
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
df.head()

Assess missing data

In [ ]:
print(df['score'].isnull().sum())

In [ ]:
print(df['comment'].isnull().sum())

In [ ]:
print(df['touchpoint'].isnull().sum())

In [ ]:
print(df['submitdate'].isnull().sum())

In [ ]:
print(df['interactionid'].isnull().sum())

In [ ]:
print(df['country'].isnull().sum())

In [ ]:
print(df['closeddate'].isnull().sum())

In [ ]:
print(df['priority'].isnull().sum())

In [ ]:
print(df['priority.1'].isnull().sum())

There is a lot of missing data - looks like there is more than 10% so multiple imputation wouldn't be recommended. 
So we will just remove all missing data.

If we had time, we could probably look at a means of identifying missing data e.g. looking at interaction IDs etc. 


In [ ]:
pd.set_option('display.max_rows', 10)
df2 = df.dropna()
display(df2)
#df2.to_csv('nomiss.csv')

## Initial Data Visualisations

In [ ]:
plt.hist(df2[['score']])

In [ ]:
plt.hist(df2['sentiment_score'])

In [ ]:
plt.hist(df2['submitdate'])

In [ ]:
plt.hist(df2['closeddate'])

In [ ]:
plt.hist(df2['created'])

## Data breakdown tables

Breakdown by Country

In [ ]:
df2.groupby(['country']).count()

Breakdown by touchpoint

In [ ]:
df2.groupby(['touchpoint']).count()

Breakdown by Priority

In [ ]:
df2.groupby(['priority']).count()

Breakdown by Client

In [ ]:
df2.groupby(['priority.1']).count()

In [ ]:
# Correlation between score and sentiment score 
# Using Spearman's Rho
spearmancorr1 = spearmanr(df2[['score', 'sentiment_score']], axis = 0)
display(spearmancorr1)
plt.scatter(df2['score'], df2['sentiment_score'])

In [ ]:
#Prediction of sentiment score based on score
reg1 = smf.ols('sentiment_score~score', data=df2).fit()
display(reg1.summary())
fig_reg1 = plt.figure(figsize=(20,20))
fig_reg1 = sm.graphics.plot_regress_exog(reg1, 'score', fig = fig_reg1)
#adj R2 = .415, F(7325, 1)=5197, p<.001, coef DV = 1.779, score = 0.2505 so if they scored 10 DV would be about 5. 

The regression shows a pretty decent model. I've put that in the notes. 

## Break the data down by date to compare scores and sentiment scores between two time periods

In [ ]:
# Extract data if the column meets the first few characters. 
# We need to convert these datetime stamps into just dates for this
df3 = df2
types = df3.dtypes
print(types)
df3.head()

In [ ]:
df3['Created_str'] = df3['created'].dt.strftime('%Y-%m-%d')

In [ ]:
display(df3)
df3.dtypes

In [ ]:
plt.hist(df3['Created_str'])
df4 = df3
df4.mar16 = df4.Created_str.astype(str).str.contains('2016-03')
#df_march16 = df3['created'].astype(str).startswith('2016-03')
df4['mar16'] = df4.mar16
display(df4)

In [ ]:
display(df4)

In [ ]:
df_Mar16 = df4[(df4['mar16'] == True)]
#display(df_Mar16)

In [ ]:
df_Mar16_ss1 = df_Mar16[(df_Mar16['sentiment_score'] == 1)]
#display(df_Mar16_ss1);
df_Mar16_ss1.to_csv('df_Mar16_ss1.csv')

In [ ]:
display(df_Mar16_ss1)
#df_Mar16_ss1.groupby(['priority']).count()

In [ ]:
df_Mar16_ss10 = df_Mar16[(df_Mar16['sentiment_score'] == 5)]
#display(df_Mar16_ss10);
df_Mar16_ss10.groupby(['priority']).count()
df_Mar16_ss10.to_csv('df_Mar16_ss10.csv')

In [ ]:
df_Mar16_ss1.to_csv('df_Mar16_ss1.csv')

In [ ]:
os.getcwd()

## Breakdown for November 2016

In [ ]:
#plt.hist(df3['Created_str'])
df4.nov16 = df4.Created_str.astype(str).str.contains('2016-11')
#df_march16 = df3['created'].astype(str).startswith('2016-03')
df4['Nov16'] = df4.nov16
#display(df4)

In [ ]:
df_Nov16 = df4[(df4['Nov16'] == True)]
#display(df_Mar16)

In [ ]:
df_Nov16_ss1 = df_Nov16[(df_Nov16['sentiment_score'] == 1)]
display(df_Nov16_ss1)
df_Nov16_ss1.to_csv('df_Nov16_ss1.csv')

In [ ]:
df_Nov16_ss10 = df_Nov16[(df_Nov16['sentiment_score'] == 5)]
#display(df_Nov16_ss10)
df_Nov16_ss10.to_csv('df_Nov16_ss10.csv')

In [ ]:
df_for_themes_bad = df_Nov16[(df_Nov16['sentiment_score'] == 1)]
df_for_themes_bad_priority_high = df_for_themes_bad[(df_for_themes_bad['priority'] == 'High')]
#display(df_for_themes_bad_priority_high)

In [ ]:
df_ss5 = df2[(df2['sentiment_score'] == 5)]
df_PH_ss5 = df_ss5[(df_ss5['priority'] == 'High')]
display(df_PH_ss5)
df_PH_ss5.to_csv('df_PH_ss5.csv')

In [ ]:
df_PC_ss5 = df_ss5[(df_ss5['priority'] == 'Critical')]
display(df_PC_ss5)
df_PC_ss5.to_csv('df_PC_ss5.csv')

In [ ]:
df_ss1 = df2[(df2['sentiment_score'] == 1)]
df_PH_ss1 = df_ss1[(df_ss1['priority'] == 'High')]
display(df_PH_ss1)
df_PH_ss1.to_csv('df_PH_ss1.csv')

In [ ]:
df_PC_ss1 = df_ss1[(df_ss1['priority'] == 'Critical')]
display(df_PC_ss1)
df_PC_ss1.to_csv('df_PC_ss1.csv')

In [ ]:
df3 = df2[df2['created'].str.contains('2016-03')]
display(df3)


In [ ]:
df2['created'].loc['2016-03']

In [ ]:
df4.groupby(df4['priority']).count()

## t-test for analysis of score based on sentiment_scores and Mar16 and Nov16

In [ ]:
#Figure it out en route home
group1 = df_Mar16['sentiment_score']
group2 = df_Nov16['sentiment_score']
from scipy.stats import ttest_ind
#perform independent two sample t-test
ttest_ind(group1, group2, equal_var = False)
#t(197)=-3.28, p<.001 - significant difference. 
#This is completely irrelevant because it's not priority, it's supposed to be the two time points.

In [ ]:
len(group1)

In [ ]:
len(group2)

In [ ]:
(len(group1) + len(group2))

In [ ]:
np.mean(group1)

In [ ]:
np.mean(group2)

In [ ]:
#Plot graphs
plt.rc('font', size=15)
#pl = 7.81
#pm = 7.97
#ph = 7.1
#pc = 8.3

priority = [3.75, 3.69]
#Day3 = [-0.26, 28.25, 3.37, 5.25, 30.87, 13.62]

x = ['March 2016', 'November 2016']
#lgnd = ['Day 1', 'Day 3']
f = plt.figure()
f.set_figwidth(8)
f.set_figheight(8)

width =0.3
plt.bar(x, priority, width=width)
#plt.bar(np.arange(len(Day3)) + width, Day3, width=width)
#plt.legend(lgnd,loc=1)
plt.title('Mean Sentiment Scores between \n March 2016 and November 2016', weight='bold')
plt.xlabel('Timepoint')
plt.ylabel('Sentiment Score')
plt.show() 

## Extraction of Data between high and critical priority and ss scores of 1 and 10

In [ ]:
#display(df2)
import statsmodels.api as sm
from statsmodels.formula.api import ols
cw_lm = ols('score ~ C(priority)', data=df2).fit()
print(sm.stats.anova_lm(cw_lm, typ=2))

In [ ]:
df_priority_low = df2[(df2['priority'] == 'Low')]
df_priority_med = df2[(df2['priority'] == 'Medium')]
df_priority_high = df2[(df2['priority'] == 'High')]
df_priority_critical = df2[(df2['priority'] == 'Critical')]

In [ ]:
df_priority_low_l = np.mean(df_priority_low.score)

In [ ]:
df_priority_low_m = np.mean(df_priority_med.score)

In [ ]:
df_priroity_high_m = np.mean(df_priority_high.score)

In [ ]:
df_priority_critical_m = np.mean(df_priority_critical.score)
print(df_priority_critical_m)

In [ ]:
#Plot graphs
plt.rc('font', size=15)
pl = 7.81
pm = 7.97
ph = 7.1
pc = 8.3

priority = [7.81, 7.97, 7.1, 8.3]
#Day3 = [-0.26, 28.25, 3.37, 5.25, 30.87, 13.62]

x = ['Low Priority', 'Medium Priority', 'High Priority', 'Critical Priority']
#lgnd = ['Day 1', 'Day 3']
f = plt.figure()
f.set_figwidth(8)
f.set_figheight(8)

width =0.3
plt.bar(x, priority, width=width)
#plt.bar(np.arange(len(Day3)) + width, Day3, width=width)
#plt.legend(lgnd,loc=1)
plt.title('Mean Scores Across Levels of Priority', weight='bold')
plt.xlabel('Priorty')
plt.ylabel('score')
plt.show() 

## Do the same but for Sentiment Score

In [ ]:
cw_lm2 = ols('sentiment_score ~ C(priority)', data=df2).fit()
print(sm.stats.anova_lm(cw_lm2, typ=2))

In [ ]:
df_priority_low_sc = np.mean(df_priority_low.sentiment_score)
df_priority_med_sc = np.mean(df_priority_med.sentiment_score)
df_priority_high_sc = np.mean(df_priority_high.sentiment_score)
df_priority_crit_sc = np.mean(df_priority_critical.sentiment_score)
print(df_priority_low_sc)

In [ ]:
print(df_priority_med_sc)

In [ ]:
print(df_priority_high_sc)

In [ ]:
print(df_priority_crit_sc)

In [ ]:
plt.rc('font', size=15)
#pl = 7.81
#pm = 7.97
#ph = 7.1
#pc = 8.3

priority = [3.75, 3.77, 3.43, 3.96]

x = ['Low Priority', 'Medium Priority', 'High Priority', 'Critical Priority']
f = plt.figure()
f.set_figwidth(8)
f.set_figheight(8)

width =0.3
plt.bar(x, priority, width=width)
plt.title('Mean Sentiment Scores Across Levels of Priority', weight='bold')
plt.xlabel('Priorty')
plt.ylabel('score')
plt.show() 

For the qualitative side of things; this might be useful 

https://towardsdatascience.com/a-beginners-guide-to-sentiment-analysis-in-python-95e354ea84f6

https://medium.com/towards-data-science/sentimental-analysis-using-vader-a3415fef7664